## Прикладная статистика в машинном обучении

### Семинар 12. Байесовские методы.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats 

In [ ]:
import jax.numpy as jnp
import jax
from jax import random, vmap
from jax.scipy.special import logsumexp

In [ ]:
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

#### Задание 0. Теория.

Основа основ -- формула Байеса:

$$
p(\theta | x) = \dfrac{p(x | \theta)p(\theta)}{p(x)}.
$$

"Апостериорное распределение вектора параметров пропорционально произведению функции правдоподобия и априорного распределения параметров."

**Простой дискретный пример.**

В Небольшом пруду водятся караси и щуки, причём согласно поверью, в любой момент времени доля карасей в Небольшом пруду может равновероятно быть либо $1/3$, либо $2/3$ в зависимости от погодных условий. Исследовательница Авдотья вылавливает десять рыб подряд. Оказалось, что в полученной выборке шесть щук и четыре карася. 

а) Найдите оценку максимального правдоподобия $\hat{p}_{c}$, рассуждая с позиций частотного подхода.


b) Теперь будем рассуждать с позиций байесовского подхода. Выпишите априорное распределение вероятности встретить карася $p_с$ и функцию правдоподобия.

c) Выпишите апостериорное распределение $\theta$ с точностью до константы.

d) Рассчитайте апостериорное распределение $\theta$.

#### Задание 1. Караси и щуки.

В Большом пруду водятся караси и щуки, причём согласно поверью, вероятность встретить карася равномерно распределена на отрезке от 0 до 1. Ловля рыбы в Большом пруду запрещена, поэтому исследовательница Авдотья решает провести численный эксперимент для вывода апостериорного распределения вероятности встретить карася.

1. Сгенерируйте выборку из из 100 наблюдений, каждое из которых является идентификатором того, является ли пойманная рыба карасём.

2. Задайте вероятностную модель.

3. Оцените модель и получите приблизительное апостериорное распределение параметров.

#### Задание 2. Структурный сдвиг.

Предположим, что следующая выборка представляет собой набор данных по числу аварий в шахтах. 

In [ ]:
years = np.arange(1800, 2000)
casual_1 = stats.poisson(3).rvs(100)
casual_2 = stats.poisson(1).rvs(100)
casual = np.hstack((casual_1, casual_2))

In [ ]:
df = pd.DataFrame({'year' : years, 'casualties' : casual})
df

In [ ]:
plt.plot(df.year, df.casualties, "o", markersize = 8, alpha=0.4)
plt.ylabel("Casualties")
plt.xlabel("Year")

In [ ]:
y = casual

Будем считать, что $x_i \sim \mathrm{Pois}(\lambda)$, где 

$$
\lambda = \begin{cases}
\lambda_1, t < \tau, \\
\lambda_2, t \ge \tau
\end{cases}
$$

Априорное распределение на $\lambda$ задаётся следующим образом: $\lambda_i \sim \mathrm{Exp}(\alpha)$, где $\alpha$ -- некоторый гиперпараметр.

а) Оцените $\alpha$ на основе выборки. Так делать не честно, но так делают.

b) Задайте вероятностную модель.

In [ ]:
def model(y):
    tau = numpyro.sample('tau', dist.Uniform(years.min(), years.max()))
    early_rate = numpyro.sample('early_rate', dist.Exponential(rate = alpha))
    late_rate = numpyro.sample('late_rate', dist.Exponential(rate = alpha))
    
    rate = early_rate
    print(type(tau))
    
    y_obs = numpyro.sample('obs', dist.Poisson(rate), obs = y)

In [ ]:
kernel = NUTS(model) # NUTS sampler 
mcmc = MCMC(kernel, 100, 500)
mcmc.run(random.PRNGKey(0), y = y)
mcmc.print_summary()
samples_1 = mcmc.get_samples()

In [ ]:
import pymc3 as pm
import arviz as az

In [ ]:
with pm.Model() as disaster_model:
    
    # априорные распределения для параметров: 
    tau = pm.DiscreteUniform("tau", lower=years.min(), upper=years.max())
    
    early_rate = pm.Exponential("early_rate", alpha)
    late_rate = pm.Exponential("late_rate", alpha)
    
    # разные значения lambda в зависимости от tau
    rate = pm.math.switch(tau >= years, early_rate, late_rate)
    
    # связь y с rate
    y_obs = pm.Poisson("disasters", rate, observed=y)

c) Оцените модель и получите приблизительное апостериорное распределение параметров.

In [ ]:
with disaster_model:
    trace = pm.sample(1000, return_inferencedata=False, model=disaster_model)

d) Оцените вероятность того, что $\lambda_i$ различны.

#### Пример. Байесовская регрессия.

В этом задании мы будем строить предсказательную байесовскую линейную регрессионную модель.

Исследуем набор данных `WaffleDivorce`, содержащий информацию о проценте разводов в 50 штатах США. 

In [ ]:
DATASET_URL = 'https://raw.githubusercontent.com/rmcelreath/rethinking/master/data/WaffleDivorce.csv'
dset = pd.read_csv(DATASET_URL, sep=';')
dset

Для построения модели отберём какие-нибудь из следующих регрессоров:
- `Population`
- `MedianAgeMarriage`
- `Marriage`
- `WaffleHouses`
- `South`.

In [ ]:
regr = ['Divorce', 'Population', 'MedianAgeMarriage', 'Marriage', 'WaffleHouses', 'South']
sns.pairplot(dset, x_vars = regr, y_vars = regr)

Замечаем, что есть корреляция между:
- `Divorce` и `Marriage`.
- `Divorce` и `MedianAgeMarriage`.

и!
- `Divorce` и `WaffleHouses` (слабая).

In [ ]:
sns.regplot('WaffleHouses', 'Divorce', dset)

Это пример ложной корреляции: скорее всего, число `Waffle House` в штате не влияет на процент разводов, но может коррелировать с другими, влияющими, факторами (какими?). 

**Вывод:** для построения модели будем использовать `Marriage` и `MedianAgeMarriage` в качестве регрессоров.

In [ ]:
X = dset[['Divorce', 'Marriage', 'MedianAgeMarriage']]

Стандартизируем переменные для лучшей сходимости.

In [ ]:
stand = lambda x: (x - x.mean()) / x.std()
X = X.apply(stand)

Рассмотрим модель вида

$$
\mathrm{divorce} \sim \mathcal{N}(\mu, \sigma^2).
$$

$$
\mu = \beta_0 + \beta_1\mathrm{marriage} + \beta_2\mathrm{age}.
$$

В `numpyro` такую модель можно задать следующим образом:

In [ ]:
def model(divorce = None, marriage = None, age = None):
    beta0 = numpyro.sample('beta0', dist.Normal(0, 0.2))
    MARRIAGE, AGE = 0, 0
    
    if marriage is not None:
        beta1 = numpyro.sample('beta1', dist.Normal(0, 0.5))
        MARRIAGE = beta1 * marriage
        
    if age is not None:
        beta2 = numpyro.sample('beta2', dist.Normal(0, 0.5))
        AGE = beta2 * age
    
    sigma = numpyro.sample('sigma', dist.Exponential(1))
    mu = beta0 + MARRIAGE + AGE
    numpyro.sample('obs', dist.Normal(mu, sigma), obs = divorce)

**Спецификация 1.** Используем в качестве регрессора только `marriage`.

In [ ]:
basic_model = pm.Model()

with basic_model:
    p = pm.Uniform(name="karases_prop", lower=0, upper=1)   # априорное распределение для доли 
    y_obs = pm.Bernoulli(name='y_obs', p = p, observed=y)   # модель

In [ ]:
kernel = NUTS(model) # NUTS sampler 
mcmc = MCMC(kernel, 1000, 2000)
mcmc.run(random.PRNGKey(0), marriage = X['Marriage'].values, divorce = X['Divorce'].values)
mcmc.print_summary()
samples_1 = mcmc.get_samples()

In [ ]:
def plot_regression(x, y_mean, y_hpdi):
    # Sort values for plotting by x axis
    idx = jnp.argsort(x)
    marriage = x[idx]
    mean = y_mean[idx]
    hpdi = y_hpdi[:, idx]
    divorce = X['Divorce'].values[idx]

    # Plot
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6))
    ax.plot(marriage, mean)
    ax.plot(marriage, divorce, 'o')
    ax.fill_between(marriage, hpdi[0], hpdi[1], alpha=0.3, interpolate=True)
    return ax

# Compute empirical posterior distribution over mu
posterior_mu = jnp.expand_dims(samples_1['beta0'], -1) + \
               jnp.expand_dims(samples_1['beta1'], -1) * X['Marriage'].values

mean_mu = jnp.mean(posterior_mu, axis=0)
hpdi_mu = hpdi(posterior_mu, 0.95)
ax = plot_regression(X['Marriage'].values, mean_mu, hpdi_mu)
ax.set(xlabel='Marriage rate', ylabel='Divorce rate', title='Regression line with 95% CI')

In [ ]:
from numpyro.infer import Predictive

predictive = Predictive(model, samples_1)
predictions = predictive(random.PRNGKey(0), marriage=X['Marriage'].values)['obs']
df = dset.filter(['Location'])
df['Mean Predictions'] = jnp.mean(predictions, axis=0)
df.head()

#### Источники мудрости.

[1] [Bayesian Regression Using NumPyro](http://num.pyro.ai/en/latest/tutorials/bayesian_regression.html)

[2] [MCMC](https://habr.com/ru/company/wunderfund/blog/279545/)